# Prerequisites:

In [10]:
!pip install scikit-image

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 3.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.9 MB 3.2 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/12.9 MB 3.9 MB/s eta 0:00:03
   ------------ --------------------------- 3.9/12.9 MB 4.3 MB/s eta 0:00:03
   ---------------- ----------------------- 5.2/12.9 MB 4.5 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.9 MB 4.9 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.9 MB 5.1 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.9 MB 5.2 MB/s eta 0:00:01
   --------------------------------- ------ 10.7/12.9 MB 5.5 MB/s eta 0:00:01
   ------------------------------------- -- 12.1/12.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 5.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
!pip install tensorflow opencv-python numpy scikit-learn

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/390.3 MB 5.6 MB/s eta 0:01:10
   ---------------------------------------- 1.0/390.3 MB 2.8 MB/s eta 0:02:20
   ---------------------------------------- 2.6/390.3 MB 4.4 MB/s eta 0:01:28
   ---------------------------------------- 3.4/390.3 MB 4.2 MB/s eta 0:01:33
   ---------------------------------------- 4.2/390.3 MB 4.1 MB/s eta 0:01:34
    --------------------------------------- 5.0/390.3 MB 4.0 MB/s eta 0:01:36
    --------------------------------------- 5.8/390.3 MB 4.2 MB/s eta 0:01:32
    --------------------------------------- 6.3/390.3 MB 4.0 MB/s eta 0:01:36
    --------------------------------------- 7.9/390.3 MB 4.3 MB/s eta 0:01:30
    --------------------------------------- 9.2/390.3 MB 4.5 MB/s eta 0:01:26
   - -------------------------------------- 10.2/390.3 MB 4.5 MB/s eta 0:01:25
   - -------------------------------------- 12.1/390.3 MB 4.9 MB/s eta

In [8]:
!pip install torch torchvision torchaudio

  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.8/204.1 MB 6.7 MB/s eta 0:00:31
   ---------------------------------------- 1.3/204.1 MB 3.4 MB/s eta 0:01:01
   ---------------------------------------- 2.4/204.1 MB 4.1 MB/s eta 0:00:50
    --------------------------------------- 3.1/204.1 MB 4.4 MB/s eta 0:00:46
    --------------------------------------- 4.2/204.1 MB 4.2 MB/s eta 0:00:48
    --------------------------------------- 4.7/204.1 MB 4.0 MB/s eta 0:00:51
   - -------------------------------------- 5.2/204.1 MB 3.8 MB/s eta 0:00:54
   - -------------------------------------- 6.3/204.1 MB 4.0 MB/s eta 0:00:50
   - -------------------------------------- 7.6/204.1 MB 4.2 MB/s eta 0:00:47
   - -------------------------------------- 8.9/204.1 MB 4.4 MB/s eta 0:00:45
   - -------------------------------------- 10.0/204.1 MB 4.5 MB/s eta 0:00:44



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Part 1: Preprocessing of Fingerprints

In [35]:
#UNET will need some data to know that an ideal mask looks like, for this I am utilising the contact based fingerprint images

In [65]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
import cv2
from sklearn.model_selection import train_test_split

def build_lightweight_unet(input_shape=(128, 128, 1)):
    inputs = Input(input_shape)
    
    # Encoder - reduced filters
    conv1 = Conv2D(8, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(16, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bridge
    conv3 = Conv2D(32, 3, activation='relu', padding='same')(pool2)
    
    # Decoder - simple upsampling
    up1 = UpSampling2D(size=(2, 2))(conv3)
    up1 = concatenate([conv2, up1], axis=3)
    conv4 = Conv2D(16, 3, activation='relu', padding='same')(up1)
    
    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = concatenate([conv1, up2], axis=3)
    conv5 = Conv2D(8, 3, activation='relu', padding='same')(up2)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def load_and_preprocess_image(path, target_size=(128, 128)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, target_size)
    return img / 255.0

def batch_load_data(contactless_path, contact_path, max_samples=1000):
    """Load limited number of samples for faster training"""
    images = []
    masks = []
    sample_count = 0
    
    user_dirs = sorted([d for d in os.listdir(contactless_path) if d.startswith('p')])
    
    for user_dir in user_dirs:
        if sample_count >= max_samples:
            break
            
        user_num = int(user_dir[1:])
        for sample_num in range(1, 4):  # Reduced from 6 to 3 samples per user
            contactless_img_path = os.path.join(contactless_path, f'p{user_num}', f'p{sample_num}.bmp')
            contact_img_path = os.path.join(contact_path, f'{user_num}_{sample_num}.jpg')
            
            if not os.path.exists(contactless_img_path) or not os.path.exists(contact_img_path):
                continue
                
            img = load_and_preprocess_image(contactless_img_path)
            mask = load_and_preprocess_image(contact_img_path)
            
            if img is not None and mask is not None:
                images.append(img)
                masks.append(mask)
                sample_count += 1
                
            if sample_count >= max_samples:
                break
    
    return np.array(images), np.array(masks)

def train_model(model, X_train, X_val, y_train, y_val):
    return model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,  # Reduced epochs
        batch_size=16,  # Smaller batch size
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
        ]
    )

def main(contactless_path, contact_path):
    print("Loading data...")
    images, masks = batch_load_data(contactless_path, contact_path, max_samples=500)
    print(f"Loaded {len(images)} image pairs")
    
    X_train, X_val, y_train, y_val = train_test_split(
        images, masks, test_size=0.2, random_state=42
    )
    
    # Add channel dimension
    X_train = np.expand_dims(X_train, axis=-1)
    X_val = np.expand_dims(X_val, axis=-1)
    y_train = np.expand_dims(y_train, axis=-1)
    y_val = np.expand_dims(y_val, axis=-1)
    
    print("Building model...")
    model = build_lightweight_unet()
    
    print("Training model...")
    history = train_model(model, X_train, X_val, y_train, y_val)
    
    print("Training completed!")
    model.save('fingerprint_model_light.keras')
    return model, history

if __name__ == "__main__":
    contactless_path = "C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\processed_contactless_2d_fingerprint_images\\first_session"
    contact_path = "C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\contact-based_fingerprints\\first_session"
    trained_model, history = main(contactless_path, contact_path)

Loading data...
Loaded 500 image pairs
Building model...
Training model...
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 192s 7s/step - accuracy: 0.5632 - loss: 0.6056 - val_accuracy: 0.5575 - val_loss: 0.5369
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 174s 7s/step - accuracy: 0.5581 - loss: 0.5269 - val_accuracy: 0.5575 - val_loss: 0.4802
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 174s 7s/step - accuracy: 0.5639 - loss: 0.4669 - val_accuracy: 0.5575 - val_loss: 0.4593
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 633s 26s/step - accuracy: 0.5603 - loss: 0.4524 - val_accuracy: 0.5575 - val_loss: 0.4516
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.5595 - loss: 0.4471 - val_accuracy: 0.5575 - val_loss: 0.4529
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.5612 - loss: 0.4461 - val_accuracy: 0.5575 - val_loss: 0.4441
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.5655 - loss: 0.4405 - val_accuracy: 0.5575 - val_loss: 0.4417
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 

In [73]:
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

def process_fingerprint(image_path, model, target_size=(128, 128)):
    """Process a single fingerprint image"""
    # Load image
    original = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if original is None:
        print(f"Failed to load image: {image_path}")
        return None, None, None
    
    print(f"Image loaded successfully. Shape: {original.shape}")
    
    # Convert to float and normalize
    image_float = original.astype(np.float32) / 255.0
    
    # Resize for model
    try:
        resized = cv2.resize(image_float, target_size, interpolation=cv2.INTER_LINEAR)
        print(f"Resized image shape: {resized.shape}")
    except Exception as e:
        print(f"Error during resize: {e}")
        return None, None, None
    
    try:
        # Prepare for model
        model_input = np.expand_dims(resized, axis=-1)
        model_input = np.expand_dims(model_input, axis=0)
        print(f"Model input shape: {model_input.shape}")
        
        # Get prediction
        mask_resized = model.predict(model_input, verbose=0)[0, :, :, 0]
        print(f"Prediction mask shape: {mask_resized.shape}")
        
        # Ensure mask is in correct format for resize
        mask_resized = mask_resized.astype(np.float32)
        
        # Resize mask back to original size with explicit checks
        h, w = original.shape
        mask = cv2.resize(mask_resized, (w, h), interpolation=cv2.INTER_LINEAR)
        
        # Apply mask to original image
        processed = image_float * mask
        
        return image_float, processed, mask
        
    except Exception as e:
        print(f"Error during prediction or processing: {e}")
        print(f"Mask resized dtype: {mask_resized.dtype}")
        print(f"Mask resized min/max: {mask_resized.min()}, {mask_resized.max()}")
        return None, None, None

def visualize_results(original_img, processed_img, mask, save_path=None):
    """Visualize results"""
    plt.switch_backend('Agg')
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(processed_img, cmap='gray')
    plt.title('Processed Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap='gray')
    plt.title('Segmentation Mask')
    plt.axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        plt.close()

def process_and_visualize_batch(model, input_dir, output_dir, num_samples=5):
    """Process and visualize a batch of fingerprint images"""
    print(f"Starting processing of {num_samples} images...")
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output directory: {output_dir}")
    
    # Get image files from nested directories
    image_files = []
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.bmp'):
                image_files.append(os.path.join(root, file))
    
    if not image_files:
        print(f"No images found in {input_dir}")
        return
        
    image_files = image_files[:num_samples]
    print(f"Found {len(image_files)} images")
    
    for idx, image_path in enumerate(image_files):
        print(f"\nProcessing image {idx + 1}/{len(image_files)}: {os.path.basename(image_path)}")
        
        # Process image
        original, processed, mask = process_fingerprint(image_path, model)
        
        if original is None:
            continue
            
        # Save results
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        
        # Save combined visualization
        viz_path = os.path.join(output_dir, f'{base_name}_combined.png')
        visualize_results(original, processed, mask, viz_path)
        print(f"Saved visualization to: {viz_path}")
        
        # Save individual images
        cv2.imwrite(os.path.join(output_dir, f'{base_name}_original.png'), 
                   (original * 255).astype(np.uint8))
        cv2.imwrite(os.path.join(output_dir, f'{base_name}_processed.png'), 
                   (processed * 255).astype(np.uint8))
        cv2.imwrite(os.path.join(output_dir, f'{base_name}_mask.png'), 
                   (mask * 255).astype(np.uint8))
        
        print(f"Successfully processed: {os.path.basename(image_path)}")

def main():
    model_path = 'fingerprint_model_light.keras'
    input_dir = r"C:\Users\SanyaKapoor\Downloads\Cross_Fingerprint_Images_Database\processed_contactless_2d_fingerprint_images\first_session"
    output_dir = r"C:\Users\SanyaKapoor\Downloads\Cross_Fingerprint_Images_Database\results"
    
    try:
        print("Loading model...")
        model = tf.keras.models.load_model(model_path)
        print("Model loaded successfully")
    except Exception as e:
        print(f"Error loading model: {e}")
        return
    
    process_and_visualize_batch(model, input_dir, output_dir, num_samples=5)

if __name__ == "__main__":
    main()

Loading model...
Model loaded successfully
Starting processing of 5 images...
Output directory: C:\Users\SanyaKapoor\Downloads\Cross_Fingerprint_Images_Database\results
Found 5 images

Processing image 1/5: p1.bmp
Image loaded successfully. Shape: (225, 350)
Resized image shape: (128, 128)
Model input shape: (1, 128, 128, 1)
Prediction mask shape: (128, 128)
Saved visualization to: C:\Users\SanyaKapoor\Downloads\Cross_Fingerprint_Images_Database\results\p1_combined.png
Successfully processed: p1.bmp

Processing image 2/5: p2.bmp
Image loaded successfully. Shape: (225, 350)
Resized image shape: (128, 128)
Model input shape: (1, 128, 128, 1)
Prediction mask shape: (128, 128)
Saved visualization to: C:\Users\SanyaKapoor\Downloads\Cross_Fingerprint_Images_Database\results\p2_combined.png
Successfully processed: p2.bmp

Processing image 3/5: p3.bmp
Image loaded successfully. Shape: (225, 350)
Resized image shape: (128, 128)
Model input shape: (1, 128, 128, 1)
Prediction mask shape: (128, 12

# Part 2 & 3: Identifying Fingerprint Orientation Fields Using a Gradient-Based Method, Minutiae Extraction

In [75]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.morphology import skeletonize
from skimage.util import invert

class FingerprintAnalyzer:
    def __init__(self, block_size=16):
        self.block_size = block_size

    def compute_orientation_field(self, image):
        """Compute orientation field using gradient-based method"""
        # Compute gradients
        gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        
        # Compute components for orientation
        gxx = gx * gx
        gyy = gy * gy
        gxy = gx * gy
        
        # Block processing
        h, w = image.shape
        orientations = np.zeros((h // self.block_size, w // self.block_size))
        
        for i in range(0, h - self.block_size, self.block_size):
            for j in range(0, w - self.block_size, self.block_size):
                gxx_block = gxx[i:i + self.block_size, j:j + self.block_size]
                gyy_block = gyy[i:i + self.block_size, j:j + self.block_size]
                gxy_block = gxy[i:i + self.block_size, j:j + self.block_size]
                
                # Compute dominant direction
                num = 2 * np.sum(gxy_block)
                den = np.sum(gxx_block - gyy_block)
                orientation = 0.5 * np.arctan2(num, den)
                
                orientations[i // self.block_size, j // self.block_size] = orientation
        
        return orientations

    def extract_minutiae(self, image, mask=None):
        """Extract minutiae points from fingerprint image"""
        # Ensure binary image
        binary = np.where(image > 127, 1, 0)
        
        # Apply skeletonization
        skeleton = skeletonize(binary)
        
        # Initialize minutiae lists
        endings = []
        bifurcations = []
        
        # Analyze 3x3 neighborhoods
        h, w = skeleton.shape
        for i in range(1, h-1):
            for j in range(1, w-1):
                if skeleton[i, j] == 1:
                    # Get 3x3 neighborhood
                    neighborhood = skeleton[i-1:i+2, j-1:j+2]
                    pixel_count = np.sum(neighborhood) - 1  # Subtract center pixel
                    
                    # Check if mask allows this point
                    if mask is not None and not mask[i, j]:
                        continue
                    
                    # Classify minutiae
                    if pixel_count == 1:
                        endings.append((j, i))  # x, y format for plotting
                    elif pixel_count == 3:
                        bifurcations.append((j, i))
        
        return endings, bifurcations

    def filter_minutiae(self, endings, bifurcations, min_distance=10):
        """Filter out spurious minutiae points"""
        # Filter endings
        filtered_endings = []
        for end in endings:
            valid = True
            # Check distance from other endings
            for other_end in filtered_endings:
                dist = np.sqrt((end[0] - other_end[0])**2 + (end[1] - other_end[1])**2)
                if dist < min_distance:
                    valid = False
                    break
            if valid:
                filtered_endings.append(end)
        
        # Filter bifurcations
        filtered_bifurcations = []
        for bif in bifurcations:
            valid = True
            # Check distance from other bifurcations
            for other_bif in filtered_bifurcations:
                dist = np.sqrt((bif[0] - other_bif[0])**2 + (bif[1] - other_bif[1])**2)
                if dist < min_distance:
                    valid = False
                    break
            if valid:
                filtered_bifurcations.append(bif)
        
        return filtered_endings, filtered_bifurcations

    def visualize_results(self, image, orientations, endings, bifurcations, save_path=None):
        """Visualize orientation field and minutiae points"""
        plt.figure(figsize=(15, 5))
        
        # Original image
        plt.subplot(131)
        plt.imshow(image, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')
        
        # Orientation field
        plt.subplot(132)
        plt.imshow(image, cmap='gray')
        h, w = orientations.shape
        y, x = np.mgrid[0:h*self.block_size:self.block_size, 0:w*self.block_size:self.block_size]
        
        # Plot orientation lines
        for i in range(h):
            for j in range(w):
                angle = orientations[i, j]
                length = self.block_size * 0.5
                dx = length * np.cos(angle)
                dy = length * np.sin(angle)
                plt.plot([x[i,j] - dx, x[i,j] + dx],
                        [y[i,j] - dy, y[i,j] + dy],
                        'r-', lw=1)
        
        plt.title('Orientation Field')
        plt.axis('off')
        
        # Minutiae points
        plt.subplot(133)
        plt.imshow(image, cmap='gray')
        if endings:
            endings = np.array(endings)
            plt.plot(endings[:, 0], endings[:, 1], 'ro', label='Endings')
        if bifurcations:
            bifurcations = np.array(bifurcations)
            plt.plot(bifurcations[:, 0], bifurcations[:, 1], 'bo', label='Bifurcations')
        plt.legend()
        plt.title('Minutiae Points')
        plt.axis('off')
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

def process_fingerprint(image_path, save_path=None):
    """Complete fingerprint analysis pipeline"""
    # Load and preprocess image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    # Initialize analyzer
    analyzer = FingerprintAnalyzer(block_size=16)
    
    # Compute orientation field
    orientations = analyzer.compute_orientation_field(image)
    
    # Extract and filter minutiae
    endings, bifurcations = analyzer.extract_minutiae(image)
    endings, bifurcations = analyzer.filter_minutiae(endings, bifurcations)
    
    # Visualize results
    analyzer.visualize_results(image, orientations, endings, bifurcations, save_path)
    
    return orientations, endings, bifurcations

def main():
    # Example usage
    image_path = "C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results\\p4_processed.png"
    save_path = "C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results"
    
    try:
        orientations, endings, bifurcations = process_fingerprint(image_path, save_path)
        print(f"Found {len(endings)} ridge endings and {len(bifurcations)} bifurcations")
    except Exception as e:
        print(f"Error processing fingerprint: {e}")

if __name__ == "__main__":
    main()

Found 86 ridge endings and 8 bifurcations


# Part 4: Matching Methods

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def visualize_core_points(img_path, core_points, title):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if isinstance(core_points, tuple):  
        core_points = [(int(core_points[0]), int(core_points[1]))]  

    plt.figure(figsize=(5,5))
    plt.imshow(img, cmap='gray')

    if core_points:
        x, y = zip(*core_points)  # Extract X and Y coordinates
        plt.scatter(x, y, c='red', marker='o', label='Core Points')

    plt.title(title)
    plt.legend()
    
    plt.savefig("core_points_image1.png")
    plt.savefig("core_points_image2.png")


# Convert core points before passing
core_points1_fixed = [(int(core_points1[0]), int(core_points1[1]))]
core_points2_fixed = [(int(core_points2[0]), int(core_points2[1]))]

# Example usage
visualize_core_points("C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results\\p4_processed.png", core_points1_fixed, "Core Points - Image 1")
visualize_core_points("C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results\\p3_processed.png", core_points2_fixed, "Core Points - Image 2")


In [95]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from scipy.spatial.distance import euclidean
from sklearn.metrics import precision_score, recall_score, f1_score

def extract_minutiae(img):
    _, binary_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    y, x = np.where(binary_img == 255)  # Minutiae points
    
    minutiae_mask = np.zeros_like(img, dtype=np.uint8)
    minutiae_mask[y, x] = 255

    img_with_minutiae = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for (px, py) in zip(x, y):
        cv2.circle(img_with_minutiae, (px, py), 2, (0, 255, 0), -1)  
    
    return list(zip(x, y)), minutiae_mask, img_with_minutiae

def compute_orientation_field(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    orientation = np.arctan2(sobely, sobelx)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(img, cmap='gray')

    step = 10
    for i in range(0, img.shape[0], step):
        for j in range(0, img.shape[1], step):
            angle = orientation[i, j]
            ax.quiver(j, i, np.cos(angle), np.sin(angle), color='r', scale=30)

    plt.savefig("orientation.png")  # Save the figure
    return orientation

def generalized_hough_matching(img1, img2):
    _, edges1, _ = extract_minutiae(img1)  # Fixed unpacking
    _, edges2, _ = extract_minutiae(img2)
    
    if edges1.shape != edges2.shape:
        return 0  # Example: Handle mismatched shapes

    match_score = np.sum(edges1 == edges2) / np.prod(edges1.shape)
    return match_score

# Method 2: Genetic Algorithm Matching
def genetic_algorithm_matching(minutiae1, minutiae2, generations=100, pop_size=50):
    def fitness(mapping):
        max_dim = max(img1.shape)  # Normalize based on image size
        return -sum(euclidean(minutiae1[i], minutiae2[mapping[i]]) / max_dim for i in range(len(minutiae1)))  

    population = [random.sample(range(len(minutiae2)), len(minutiae1)) for _ in range(pop_size)]
    for _ in range(generations):
        population.sort(key=fitness, reverse=True)
        new_population = population[:10] + [random.sample(range(len(minutiae2)), len(minutiae1)) for _ in range(pop_size-10)]
        population = new_population

    return -fitness(population[0])

# Method 3: Core Point Matching
def core_point_matching(img1, img2):
    core1 = np.unravel_index(np.argmax(img1), img1.shape)
    core2 = np.unravel_index(np.argmax(img2), img2.shape)
    shift = np.array(core2) - np.array(core1)
    img1_shifted = np.roll(img1, shift, axis=(0, 1))
    return np.sum(img1_shifted == img2) / np.prod(img1.shape)

# Performance Metrics Calculation
def evaluate_performance(scores):
    # Example: Assume binary classification where a score > threshold is a "match"
    threshold = 0.5  
    y_true = [1, 1, 1]  # Assuming all methods should ideally match
    y_pred = [1 if score > threshold else 0 for score in scores]

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return precision, recall, f1

import matplotlib.pyplot as plt

def visualize_matching_results(img1, img2, img1_with_minutiae, img2_with_minutiae):
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))

    axs[0, 0].imshow(img1, cmap='gray')
    axs[0, 0].set_title("Original Fingerprint 1")

    axs[0, 1].imshow(img2, cmap='gray')
    axs[0, 1].set_title("Original Fingerprint 2")

    axs[1, 0].imshow(img1_with_minutiae)
    axs[1, 0].set_title("Minutiae Extracted (Fingerprint 1)")

    axs[1, 1].imshow(img2_with_minutiae)
    axs[1, 1].set_title("Minutiae Extracted (Fingerprint 2)")

    plt.savefig("final.png")

from sklearn.metrics import precision_recall_fscore_support

def compute_performance(true_labels, predicted_labels):
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')
    return precision, recall, f1

if __name__ == "__main__":
    img1 = cv2.imread('C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results\\p4_processed.png', cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread('C:\\Users\\SanyaKapoor\\Downloads\\Cross_Fingerprint_Images_Database\\results\\p3_processed.png', cv2.IMREAD_GRAYSCALE)

    # Extract minutiae and overlay
    minutiae1, _, img1_with_minutiae = extract_minutiae(img1)
    minutiae2, _, img2_with_minutiae = extract_minutiae(img2)

    # Compute orientation field
    orientation1 = compute_orientation_field(img1)
    orientation2 = compute_orientation_field(img2)

    # Compute Matching Scores
    score1 = generalized_hough_matching(img1, img2)
    score2 = genetic_algorithm_matching(minutiae1, minutiae2)
    score3 = core_point_matching(img1, img2)

    print(f"Hough Matching Score: {score1}")
    print(f"Genetic Algorithm Matching Score: {score2}")
    print(f"Core Point Matching Score: {score3}")

    # Visualize comparison
    visualize_matching_results(img1, img2, img1_with_minutiae, img2_with_minutiae)

C:\Users\SanyaKapoor\AppData\Local\Temp\ipykernel_21564\1004493056.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show(block=True)  # Ensure interactive mode


Hough Matching Score: 0.9474031746031746
Genetic Algorithm Matching Score: 587.8766747705599
Core Point Matching Score: 0.11812063492063492
